*  This program simulates, for arbitrarily given N (# of assets) and T (sample * size, a set of true mean and covariance parameters; then a set of N asset retturns drawn from a normal distribution with those parameters, and then see how well for estimated optimal portfolio performs.
***
The purpose is to understand how estimation errors affect the performance.
 

In [36]:
import numpy as np
import scipy.linalg as la      # we need the package to compute the Cholesky decomposition

np.random.seed(1234)         # so that the random numbers will be the same each time running the program

# We set the parameters first. 
# Although one can enter parameters manually, below is a simple sysmatic way
# to generate them

N = 20    # an arbitrary number of assets 
T = 120    # sample size T>N, such as 10 years of month data

mu = np.random.randn(N,1)
mu = np.exp(mu)/10  # get a vector of positive expected return parameters
                       # dividing by 100 is just to make it smaller
    
L = np.random.randn(N,N)  # get a random matrix
V = np.matmul(L.T,L)    # V = L'*L,  our covariance matrix
                         # it will be positive definite prob 1
                       #  b/c L is nonsingular with probability 1

# Assume the riskfree rate is zero
# get the theoretical Sharpe ratio from formula (2.60) of the Lecture Notes
#          

VI = np.linalg.inv(V)    # The inverse of S
                         # np.linalg.inv is a np function 

B = np.matmul(VI, mu)       #  the vector VI*mu

SR = np.matmul(mu.T,B)       #   mu'*VI*mu
     
# SR = np.sqrt(SR)

print(' The # of assets and sample size  \n')
print('          {}  {} \n'.format(N, T))   
print('  The theoretical Sharpe ratio given the parameters  \n')
print(SR)

 The # of assets and sample size  

          20  120 

  The theoretical Sharpe ratio given the parameters  

[[1.41671171]]


In [37]:
# 
# Now we simulate an initial data/returns from the normal(mu, V)
#      sample size T

L1 = la.cholesky(V)             # Cholesky decomposition:  V = L1'*L1
L = L1.T                         # V = L*L',  L=L1'

R = np.ones((N, T))    # define the matrix to store returns

for i in range(T):
    e = np.random.randn(N,1) # standard normal
    Y = mu + np.matmul(L,e)  # Y = mu + L*e, a draw from normal(mu,V)
    R[:,i]= Y.T
  
# #  Note   
#  A one-step matrix multiplication method without the loop is to use
# formula:    R = mu*1_T + L*E,  E is NxT matrix of normal(0,1)
#  The loop approach is easier to understand

In [38]:
# Compute the estimated optimal portfolio weights

mu5 = np.mean(R, axis = 1)  # the mean taking each row of the matrix,
                            # note R is NxT here  
mu5 = mu5.T               # make it a column vector
S = np.cov(R)             # the covariance estimate, N by N 

SI = np.linalg.inv(S)                     # The inverse of V

# The optimal weights on the N risky aasets

gamma = 3                             # The risk-averse coeff.

w = 1/gamma*np.matmul(SI, mu5)

# print('  the estimated optimal portfolio weights  \n')
# print(w)

In [39]:
##  Now apply the optimal estimated rule to M=100,000 random N(mu, V) returns
        
M = 100000
Z = np.ones((M,1))    # define Z to store the returns   

L1 = la.cholesky(V)             # Cholesky decomposition:  V0 = L1'*L1
L = L1.T                         # V = L*L',  L=L1'

for i in range (M):
    e = np.random.randn(N,1)    
    Y = mu + np.matmul(L,e)         # Y = L*e, a draw from normal(mu, V)
    Z[i,:] = np.dot(w, Y)          # portfolio return

# Compute the Sharpe ratio over the M draws, which is the expected Sharpe ratio 
# of the estmated optimal portfolio rule

mu1 = Z.mean()                          # The expected mkt excess return
sig2 = Z.var()                         # The var of the mkt excess return
sigma = np.sqrt(sig2)                    # Its vol

SRe =  mu1 / sigma     # The optimal weight on mkt

print(' The # of assets and sample size  \n')
print('          {}  {} \n'.format(N, T))   
print('  The theoretical Sharpe ratio and that of the estimated rule  \n')
print(SR,SRe)
print('  The percentage difference achieved  \n')
print(SRe/SR)

 The # of assets and sample size  

          20  120 

  The theoretical Sharpe ratio and that of the estimated rule  

[[1.41671171]] 1.0112810744127174
  The percentage difference achieved  

[[0.71382277]]
